In [14]:
%load_ext autoreload
%autoreload 2
%xmode Verbose
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from multiprocessing import Pool
import sys
dirr = "../../src"
if dirr not in sys.path:
    sys.path.append(dirr)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Exception reporting mode: Verbose


In [2]:
data = pd.read_excel("../../data/real_estate/dataset.xlsx")
data.head(3)

,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
0,1,2012.916667,32.0,84.87882,10,24.98298,121.54024,37.9
1,2,2012.916667,19.5,306.59470,9,24.98034,121.53951,42.2
2,3,2013.583333,13.3,561.98450,5,24.98746,121.54391,47.3


In [11]:
from rf import RandomForestRegression

In [10]:
rf = RandomForestRegression()
rf.train(data)

In [15]:
rf.predict(data.iloc[:10, :])

PicklingError: Can't pickle <class '__main__.RandomForestRegression'>: it's not the same object as __main__.RandomForestRegression

In [9]:
for d in rf.roots:
    print (helper.parser(d))

Question: Is 6th Column >= 121.55174
  T: Question: Is 0th Column >= 308
    T: Predict: 24.7
    F: Predict: 30.6
  F: Question: Is 3th Column >= 1013.341
    T: Predict: 27.3
    F: Predict: 40.3

None
Question: Is 6th Column >= 121.55174
  T: Question: Is 0th Column >= 308
    T: Predict: 24.7
    F: Predict: 30.6
  F: Question: Is 3th Column >= 1013.341
    T: Predict: 27.3
    F: Predict: 40.3

None
Question: Is 3th Column >= 837.7233
  T: Question: Is 0th Column >= 94
    T: Predict: 24.7
    F: Predict: 27.0
  F: Question: Is 2th Column >= 7.1
    T: Predict: 42.0
    F: Predict: 54.4

None
Question: Is 3th Column >= 837.7233
  T: Question: Is 0th Column >= 94
    T: Predict: 24.7
    F: Predict: 27.0
  F: Question: Is 2th Column >= 7.1
    T: Predict: 42.0
    F: Predict: 54.4

None


In [7]:
def foo(i):
    print (i)
    return i

p = mp.Process(target=foo, args=(3,))
p

NameError: name 'mp' is not defined

In [ ]:
p.start()

In [ ]:
random.seed(123)

# Define an output queue
output = mp.Queue()

# define a example function
def rand_string(length, output):
    """ Generates a random string of numbers, lower- and uppercase chars. """
    rand_str = ''.join(random.choice(
                        string.ascii_lowercase
                        + string.ascii_uppercase
                        + string.digits)
                   for i in range(length))
    output.put(rand_str)

# Setup a list of processes that we want to run
processes = [mp.Process(target=rand_string, args=(5, output)) for x in range(4)]

# Run processes
for p in processes:
    p.start()

# Exit the completed processes
for p in processes:
    p.join()

# Get process results from the output queue
results = [output.get() for p in processes]

print(results)

In [ ]:
mp.Queue()

---
## Question 4
Generating a dataset with random noise as Gaussian Distribution.

In [ ]:
def dataset_generator(m, c, samples = 50, sigma=10): 
    """assumes random noise comes from
    Gaussian Distribution
    """
    X = np.arange(samples)
    random_noise = np.random.normal(scale=sigma, size=samples)
    Y = m*X + c + random_noise
    return X, Y

In [ ]:
X, Y = dataset_generator(m=2, c=43, samples=50, sigma=10)
plt.scatter(X, Y)

---
## Question 5
Look at the code in ``utils.py`` for the algorithm used. Basically I used Linear congruential generator.

In [ ]:
import utils as helper
y = helper.random_list(N = 100, samples = 10000)
plt.hist(y)